# tutorial 4: in-pipeline hyperparameter screens

This tutorial show you how to use the **GridSearchEnsemble** class to 

* screen ML model hyperparameters during model fitting
* make ensemble predictions using the results of a hyperparameter screen

## 

In [12]:
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import pipecaster as pc

screen_specs = {
     'learning_rate':[0.1, 10],
     'n_estimators':[2, 10],
}

X, y = make_classification()
clf = pc.GridSearchEnsemble(
                 param_dict=screen_specs,
                 base_predictor_cls=GradientBoostingClassifier,
                 meta_predictor=pc.SoftVotingClassifier(),
                 internal_cv=5, scorer='auto',
                 score_selector=pc.RankScoreSelector(k=2),
                 base_processes='max')
clf.fit(X, y)
clf.get_results_df()

,selections,parameters
score,,
0.730769,+,"{'learning_rate': 0.1, 'n_estimators': 10}"
0.667468,+,"{'learning_rate': 0.1, 'n_estimators': 2}"
0.513622,-,"{'learning_rate': 10, 'n_estimators': 2}"
0.499199,-,"{'learning_rate': 10, 'n_estimators': 10}"


In [13]:
cross_val_score(clf, X, y, scoring='balanced_accuracy', cv=5)
# output: array([0.9 , 0.85, 0.85, 0.85, 0.65])

array([0.45     , 0.55     , 0.65     , 0.8989899, 0.7979798])